## Step 1: Import some packages 

As in often the case, we start our code by importing some Python modules. 

In this case, we import the objects module which holds a number of helper function that I prepared.

In [ ]:
# 'objects' module has code that I wrote to analyze this dataset
import objects
# matplotlib.pyplot module has functions for plotting, note that we are importing it as plt
import matplotlib.pyplot as plt
# import numpy for working with matrices
import numpy as np

## Step 2: Get the data from Open Science Foundation
The data files are shared in an public repository at the [Open Science Foundation](https://osf.io/bwntz/]). 

The code below calls a function

    objects.fetch_data()

that grabs the data and makes it accessible to the current workbook. 

If you inspect the function (look under "fetch_data" in objects.py) you will notice the structure of the code

    !osf --project bwntz clone
    
- "!" indicates that this we are running something from the command line, outside Python.
- "bwntz" is the ID of the project on OSF.
- "clone" simply means we want to make a local copy of the files from OSF

The command will place the files in the local folder 

    bwntz/osfstorage/data
    
which is where we grab the data from in the next step. The code also checks if files have already been downloaded, and if they have, it will not waste time redownloading.

A limitation that I am still trying to work out is that you cannot specify a subset files that you want to grab, you have to get the whole thing.

In [ ]:
project_id = "bwntz"
objects.fetch_data(project_id)

## Step 3: Load and organize the data from each participant

The code below loads in the individual data files in csv format that each participant, including you, created when they did the experiment. When running an experiment online, you would normally have some sort of back-end that saves the files automatically somewhere, so you do not have to trust your participant to send you the raw data.

In [ ]:
data_dir = "bwntz/osfstorage/data"
labels = ['bear','elephant','person','car','dog','apple','chair','plane','bird','zebra']
all_data, sub_list, conf_labels = objects.load_data(data_dir, grab_course = "nrsc2200", grab_term = "W2025", labels=labels)

## Step 4: Average confusion matrices and accuracies over participants
You may re-run this code after removing subjects from all_conf and all_corr that more than two standard errors away from the mean. 

In [ ]:
mean_conf, mean_corr, all_conf, all_corr = objects.confusion_averages(all_data)

## Step 5: Plot accuracies for each image pair
Now we will plot the accuracies for each image pair. We will use the values from the confusion matrix, but convert them into a list. 

In [ ]:
conf_list, label_list = objects.confusion_list(mean_conf, conf_labels)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 12}
plt.rc('font', **font)
plt.plot(conf_list)
ax.set_xticks(range(0,len(label_list)+1,10))
#ax.set_xticklabels(label_list[0:10:],rotation = 45, ha="center")
plt.xlabel("image pair")
plt.ylabel("accuracy")
plt.show()

## Step 6: Create a confusion matrix
The code below makes a confusion matrix. Each cell reflects the proportion of trials where the non-target image (distractor) was chosen for each image pair. Since the target and distractor were always different, the diagonal is colored white and marked with "NA". 

In [ ]:
# reorder the labels to reveal category-specific confusion
new_labels = ['bear','bird','dog','elephant','zebra','person','apple','car','chair','plane']
plt = objects.confusion_plot(mean_conf, labels, new_labels)
plt.show()

## Assignment 4

### Question 1 (4 pts)

In the previous assignment you identified outlier timepoints. In some cases, it may be similarly useful to identify outlier participants. This is especially true for behavioral experiments that are collected online, where you may have participants that do not understand the task, or simply decide not to do it. Here we will identify participants whose overall accuracy is substantially lower than the mean. 

**(A)** Write a function called <span style="color: orange; font-weight:bold">quality_control</span> that takes *all_corr* as input and returns *reject_idx*, an one-dimensional numpy array that has *True* for any participant whose accuracy is more than 2 standard errors **lower** than the mean, and *False* otherwise. *reject_idx* should have n elements, where n is the number of participants. 

**(B)** Pass that variable as the second input to the function <span style="color: green; font-weight:bold">objects.confusion_averages</span> to recompute the variables *mean_conf*, *mean_corr*, *all_conf* and *all_corr* as in Step 4 above.

**(C)** Use the recomputed average to plot the confusion matrix - this can be done by re-running <span style="color: green; font-weight:bold">objects.confusion_plot</span> from **Step 6** with the recomputed *mean_conf* - the other input variables can stay the same. Take a screenshot of your confusion matrix.

**(D)** Write a function called <span style="color: orange; font-weight:bold">rejected_ids</span> that takes *sub_list* and *reject_idx* as input and returns a variable *outliers*, which should be a one-dimensional numpy array with the IDs of the subjects that were rejected. 

**In your submitted assignment, please share the following and only the following: The code used in B and C, including your call to the quality_control function. The screenshot of the confusion matrix generated in C. Your call to the rejected_ids function and the IDs of the rejected participants. Please submit the functions created for A and D, for checking using VPL.**


In [ ]:
# work on your answer here

# (A)
def quality_control(all_corr):
    # your code here
    return(reject_idx)

# (B)

# (C)

# (D)
def rejected_ids(sub_list, reject_list):
    # your code here
    return(outliers)



### Question 2 (2 pts)

One of key points of the experiment and the paper *Simple Learned Weighted Sums of Inferior Temporal Neuronal Firing Rates Accurately Predict Human Core Object Recognition Performance* by Majaj and colleagues (2015), is that some objects are more difficult to distinguish than others. Here we will identify those objects.

**(A)** Pay attention **Step 5: Plot accuracies for each image pair**. Note that the variable *conf_list* is a sorted list of accuracies for each image pair. Also note that *label_list* is a matched list of image pair labels in the format "tt_vs_dd" where tt is the target and dd is the distractor. Write a function <span style="color: orange; font-weight:bold">image_extremes</span> that takes *conf_list* and *label_list* as input and returns the image pairs with the lowest and highest average accuracy, i.e. the distractors that were easiest and most difficult to distinguish from the targets. The two outputs should be string variables. 

**(B)** Find the image pairs you identified in the confusion matrix plotted in **Step 6**. Are your findings consistent with the confusion matrix? Explain why.

**In your submitted assignment, please share the following and only the following: The code used in A, including your call to the image_extremes function and a print statement printing the two outputs, as well as the result of that print statement. Your answer to B, in 3-5 sentences or so. Please submit your function created for B, for checking using VPL.**

In [ ]:
# work on your answer here:
def image_extremes(conf_list, label_list):
    # your code here
    return(min_pair, max_pair)

### Question 3 (4 pts)

It is often informative to establish the consistency of your measurement across participants. Here we will do something like this, by computing the split-half correlation across participants. In this assignment you will create a 3 functions that can be tested on eClass, and there will be a short answer question as well. 

**(A)** The variable *all_conf_new* holds the confusion matrices for each participant. It is a n x 10 x 10 matrix, where n is the remaining number of subjects after removing outlier participants in **Question 1**. Write a function <span style="color: orange; font-weight:bold">split_half</span> that takes a three-dimensional numpy array like that as input and returns *c1* and *c2*, two average confusion matrices, one computed over the first half of participants and one over the second half. Because the confusion matrices contain missing values, NaNs, you have to use the command **np.nanmean** to compute the averages. Both outputs (*c1* and *c2*) should be 10 x 10 numpy arrays.

**(B)** Write a function, <span style="color: orange; font-weight:bold">compute_corr</span>, that takes two average confusion matrices (e.g., *c1* and *c2*) as input and uses the function **np.corrcoef** to compute the Pearson correlation between those two confusion matrices. The function should return the output of np.corrcoef as a variable *p_c*. *p_c* should be a 2 x 2 numpy arrays (see next question). Note that your average confusion matrices will contain NaNs, so you have to remove them before applying np.corrcoef. Assuming your average is stored in variable c1, this can be done with the following code:

        c1[~np.isnan(c1)]
   
**(C)** **np.corrcoef** returns a 2 x 2 numpy array, with two values being equal to 1. The other two values are identical and both equal the Pearson correlation. Write a function <span style="color: orange; font-weight:bold">grab_corr</span> that takes the *p_c* as input and returns a single float number with the correlation. Use your functions to print the correlation. 

**(D)** In the Majaj paper the median correlation across human participants in a similar task was 0.929. Now, it is certainly possible that we simply took the task less seriously than Majaj's participants, but if we give ourselves some credit and assume that this is not the case, what are some other plausible reasons our split-half correlation was worse? 

**In your submitted assignment, please share the following and only the following: The code used in C, including your calls to split_half, compute_corr and grab_corr, as well as the print statement for printing the correlation, and the result of that statement. Your answer to D, in 5 sentences or so. Please submit your functions created for A, B and C, for checking using VPL.**

In [ ]:
# work on your answer here:
# (A)
def split_half(all_conf):
    # your code here
    return(c1, c2)

# (B)
def compute_corr(c1, c2):
    # your code here
    return(p_c)

# (C)
def grab_corr(p_c):
    # your code here
    return(out)